In [1]:
from src.metagpt import BioformatsReader
import javabridge
import bioformats
import ome_types

javabridge.start_vm(class_path=bioformats.JARS)

image_path = "/home/aaron/Documents/Projects/MetaGPT/in/images/Image_8.czi"
ome_xml = BioformatsReader.get_omexml_metadata(image_path)
ome_raw = BioformatsReader.get_raw_metadata(image_path)
ome_tree = BioformatsReader.raw_to_tree(ome_raw)
ome_dict = ome_types.to_dict(ome_xml)
print(ome_dict)

javabridge.kill_vm()

17:08:12.321 [Thread-0] DEBUG loci.common.NIOByteBufferProvider - Using mapped byte buffer? false
17:08:12.337 [Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader
java.lang.ClassNotFoundException: loci.formats.in.URLReader
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:331)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:264)
	at loci.formats.ClassList.parseLine(ClassList.java:196)
	at loci.formats.ClassList.parseFile(ClassList.java:258)
	at loci.formats.ClassList.<init>(ClassList.java:138)
17:08:12.340 [Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.SlideBook6Reader
java.lang.ClassNotFoundException: loci.formats.in.SlideBook6Reader
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lan

In [6]:
from ome_types._autogenerated.ome_2016_06 import OME
from ome_types._autogenerated.ome_2016_06 import Pixels
from pydantic import BaseModel
from typing import Optional
from marvin.beta import Application

app = Application(
    name='OME Metadata Store',
    instructions=("Incoming metadata will be provided in raw format, that mean a list of key value pairs.!!!"
                  "Your task  will be to translate these key value pairs to the appropriate OME schema property(try to figure out which porperty is which by looking at the schema and the raw metadata in holistic manner). You will be "
                  "handed only a part of the ome schema to fill in, to reduce the scope."),
    state=OME(),
)

In [8]:
print(app.thread)

AttributeError: 'Application' object has no attribute 'thread'

In [ ]:
class ToDo(BaseModel):
    name: str
    done: bool = False

class ToDoState(BaseModel):
    todos: list[ToDo] = []


# --- create the application
todo_app = Application(
    name="ToDo App", instructions="A todo application", state=ToDoState()
)


In [ ]:
class MaybePixels(BaseModel):
    pixels: Optional[Pixels] = None

app = Application(
    name='OME Metadata Store',
    instructions=("Incoming metadata will be provided in raw format, that mean a list of key value pairs.!!!"
                  "Your task  will be to translate these key value pairs to the appropriate OME schema property(try to figure out which porperty is which by looking at the schema and the raw metadata in holistic manner). You will be "
                  "handed only a part of the ome schema to fill in, to reduce the scope.Since this is a hard problem I will need you to think step by step and use chain of thought to solve this problem. Here are some example on how to approach it:"
                  "Since the tool usage requires a python middle layer the usually used property names are written a bit differently, for example SizeX is written as size_x etc. The validation tool will make you aware of the correct property names."
                  "1. Look at the schema which elements are missing which one do you plan to add"
                  "2. Choose the easiest to implement element to start, Figure out if they have mandatory fields or dependencies. Try to keep the scope per change as small as possible so validation is easier."
                  "3. Look at the raw metadata, speicifcally try to find if any of the raw metadata would fit into the previously identified fields"
                  "4. Come to a conclusion wheter you can add the element or if there are mandatory fields missing"
                  "5. If you cant add the element start from step 1 again"
                  "6. If you can add the element, generate the tool call to add the element, make sure to add the element and all the mandatory fields to not get a validation error. Think about which operation is best fitting, are you addding a new node or modifying an existing one=?"
                  "7. Call the tool"
                  "8. If validation errors occur approach them systematically, is there a mandatory node missing? Is it possible to split the tool call into several steps? is there a dependency missing? is the value of the node correct? If you cant solve the problem start from step 1 again"
                  "9. Repeat the process until all elements are added and the metadata is valid. Remember to solve this problem step by step and use chain of thought to solve it. Good luck!"
                  ),
    state=OME()
)

In [ ]:
from ome_types import OME
from ome_types import Pixels

class MaybePixels(BaseModel):
    pixels: Optional[Pixels] = None

In [ ]:
MaybePixels().model_json_schema()

In [ ]:
await app.say_async("here is the raw metadata: {}".format(ome_raw))

In [ ]:
app.state.update_state_jsonpatches([{"op": "add",
                                        "path": "",
                                        "value": {
                                            "id": "Pixels:0",
                                            "SizeX": 512,
                                            "SizeY": 512,
                                            "SizeZ": 1,
                                            "SizeC": 1,
                                            "SizeT": 1,
                                            "DimensionOrder": "XYZCT",
                                            "PixelType": "uint8",
                                            "BigEndian": False}}])

In [ ]:
myome =OME()


In [ ]:
myome.images.append(Image(Pixels(PixelsType="uint8", SizeX=512, SizeY=512, SizeZ=1, SizeC=1, SizeT=1, DimensionOrder="XYZCT", BigEndian=False)))

In [ ]:
from ome_types._autogenerated.ome_2016_06 import Image
from ome_types._autogenerated.ome_2016_06 import Pixels
Pixels(id="Pixel1", dimension_order="XYZCT", type="uint8", size_x=512, size_y=512, size_z=512, size_c=1, size_t=1).model_dump_json()

In [ ]:
# Answer App
app_answer = Application(
    name='DataProviderGPT',
    instructions="You have a dictinoary of raw metadata as your state, your task will be to answer questions about the metadata.",
    state=ome_tree,
)




In [ ]:
app_answer.say("Does the metadata contain info about an image?")

In [ ]:
a = []
def test(a=0, b=0, c=0):
    return a + b + c

test(*a)

In [ ]:
# Define your Pydantic models
class Pixels(BaseModel):
    dimension_order: str
    type: str

class Image(BaseModel):
    pixels: Pixels
    name: str

class Experimenter(BaseModel):
    name: Optional[str] = None
    email: Optional[str] = None

class OME(BaseModel):
    images: List[Image]
    experimenters: List[Experimenter]

In [ ]:
from typing import List, Dict, Set, Type
from pydantic import BaseModel, Field
from marvin.beta import Application
from collections.abc import Iterator

promp1="""
Incoming metadata will be provided in raw format, that mean a list of key value pairs.!!!
Your task  will be to translate these key value pairs to the appropriate OME schema property(try to figure out which porperty is which by looking at the schema and the raw metadata in holistic manner). You will be
handed only a part of the ome schema to fill in, to reduce the scope.Since this is a hard problem I will need you to think step by step and use chain of thought to solve this problem. Here are some example on how to approach it:
Since the tool usage requires a python middle layer the usually used property names are written a bit differently, for example SizeX is written as size_x etc. The validation tool will make you aware of the correct property names.
1. Look at the schema which elements are missing which one do you plan to add
2. Figure out if they have mandatory fields or dependencies
3. Look at the raw metadata, speicifcally try to find if any of the raw metadata would fit into the previously identified fields
4. Come to a conclusion wheter you can add the element or if there are mandatory fields missing
5. If you cant add the element start from step 1 again
6. If you can add the element, generate the tool call to add the element, make sure to add the element and all the mandatory fields to not get a validation error. Think about which operation is best fitting, are you addding a new node or modifying an existing one=?
7. Call the tool
8. If validation errors occur approach them systematically, is there a mandatory node missing? is there a dependency missing? is the value of the node correct? If you cant solve the problem you can start from step 1 again
9. Repeat the process until all elements are added and the metadata is valid. Remember to solve this problem step by step and use chain of thought to solve it. Good luck!
I have split the ome model into several submodels, so if you feel like therer is not metadata for the data model you are working on, no worries just skip that model. Its very well possible some fields remain empty.
"""

# Define the tree node class
class TreeNode:
    def __init__(self, model: Type[BaseModel]):
        self.model = model
        self.object = None
        self.children = []

    def add_child(self, child: 'TreeNode'):
        self.children.append(child)

    def __repr__(self):
        return f"TreeNode(model={self.model.__name__}, children={self.children})"
    
    
    def required_fields(self, model: type[BaseModel], recursive: bool = False) -> Iterator[str]:
        """
        https://stackoverflow.com/questions/75146792/get-all-required-fields-of-a-nested-pydantic-model
        """
        for name, field in model.model_fields.items():
            if not field.is_required():
                continue
            t = field.annotation
            if recursive and isinstance(t, type) and issubclass(t, BaseModel):
                yield from self.required_fields(t, recursive=True)
            else:
                yield name
    
    def instantiate_model(self, child_objects) -> BaseModel:
        if any(self.required_fields(self.model)):
            print(any(self.required_fields(self.model)))
            # Create a MaybeModel class with a dynamic name
            maybe_model_name = f"Maybe{self.model.__name__}"
            #maybe_object_name = f"maybe{self.model.__name__}"

            annotations = {self.model.__name__: Optional[self.model]}
            MaybeModel = type(maybe_model_name,
                              (BaseModel,),
                              {'__annotations__': annotations,
                               self.model.__name__: Field(default=None,
                                                          description="The actual object to be filled with metadata.")})

            self.object = MaybeModel()  # we will need to fill in child objects later
        else:
            print(any(self.required_fields(self.model)))
            self.object = self.model(**child_objects)
            # print what kind of object this is
            print(self.object.model_dump_json())

    def predict_meta(self, raw_meta) -> BaseModel:
        # add the metdata from the child nodes first
        child_objects = {}
        for child in self.children:
            child_objects[child.model.__name__] = child.predict_meta(raw_meta)
        
        self.object = self.instantiate_model(child_objects)
        
        print(f"Predicting metadata for {self.model.__name__}, self.object={type(self.object)}, required={list(self.required_fields(self.model))}")
        return self.object
        app = Application(
            name='OME Metadata Store',
            instructions=(promp1),
            state=self.object,
        )
        app.say("here is the raw metadata: {}".format(raw_meta))
        
        return self.object

# Function to identify dependencies and collect all models
def collect_dependencies(model: Type[BaseModel], known_models: Dict[str, Type[BaseModel]], collected: Dict[str, Type[BaseModel]]):
    if model.__name__ in collected:
        return
    collected[model.__name__] = model
    for field in model.model_fields.values():
        field_type = field.annotation
        if hasattr(field_type, '__fields__'):  # If the field is a Pydantic model
            collect_dependencies(field_type, known_models, collected)
        elif hasattr(field_type, '__origin__') and field_type.__origin__ is list:
            item_type = field_type.__args__[0]
            if hasattr(item_type, '__fields__'):
                collect_dependencies(item_type, known_models, collected)

# Function to create the tree
def create_dependency_tree(model: Type[BaseModel], known_models: Dict[str, Type[BaseModel]], visited: Set[str]) -> TreeNode:
    node = TreeNode(model)
    visited.add(model.__name__)
    
    for field in model.model_fields.values():
        field_type = field.annotation
        if hasattr(field_type, '__fields__') and field_type.__name__ not in visited:  # If the field is a Pydantic model
            child_node = create_dependency_tree(field_type, known_models, visited)
            node.add_child(child_node)
        elif hasattr(field_type, '__origin__') and field_type.__origin__ is list:
            item_type = field_type.__args__[0]
            if hasattr(item_type, '__fields__') and item_type.__name__ not in visited:
                child_node = create_dependency_tree(item_type, known_models, visited)
                node.add_child(child_node)
    
    return node

# Function to build the tree starting from the root model
def build_tree(root_model: Type[BaseModel]) -> TreeNode:
    known_models = {model.__name__: model for model in globals().values() if isinstance(model, type) and issubclass(model, BaseModel)}
    collected_models = {}
    collect_dependencies(root_model, known_models, collected_models)
    
    return create_dependency_tree(root_model, known_models, set())

# Build the tree starting from the OME model
dependency_tree = build_tree(OME)

# Print the tree structure
def print_tree(node: TreeNode, indent: str = ""):
    print(f"{indent}{node.model.__name__}")
    for child in node.children:
        print_tree(child, indent + "  ")

print_tree(dependency_tree)


In [ ]:
dependency_tree.predict_meta(ome_raw)

In [ ]:
dependency_tree.children

In [ ]:
from ome_types import OME

dependency_tree = build_tree(OME)

dependency_tree.predict_meta(ome_raw)

In [ ]:
from ome_types import OME
from ome_types._autogenerated.ome_2016_06 import Image
from ome_types._autogenerated.ome_2016_06 import Pixels

myome = OME(images=[Image(pixels=Pixels(dimension_order='XYZCT', type='uint8', size_x=512, size_y=512, size_z=1, size_c=1, size_t=1))])

app = Application(
    name='OME Metadata Store',
    instructions=(promp1),
    state=myome,
)
app.

In [ ]:
app.state.value.images.append(Image(pixels=Pixels(dimension_order='XYZCT', type='uint8', size_x=512, size_y=512, size_z=1, size_c=1, size_t=1)))

In [ ]:
pix = Pixels(dimension_order='XYZCT', type='uint8', size_x=512, size_y=512, size_z=1, size_c=1, size_t=1)

In [ ]:
Image.model_fields

In [ ]:
pix_dic = {type(pix).__name__.lower(): pix}
pix_dic
img = Image(**pix_dic)
ome = OME()


In [ ]:
obj = None
if obj:
    print("yes")

In [ ]:
def add_to_instance(instance, obj):
    mod_dic = {}
    # Get the type of the object
    obj_type = type(obj)
    
    # Define a set of ignored types
    ignored_types = {str, int, float, bool} # could be updated to != ome_types
    # Skip ignored types
    if obj_type in ignored_types:
        return
    # Iterate over the attributes and their types in the instance's class
    for attr_name, attr_type in instance.__annotations__.items():
        
        if attr_type == Optional[obj_type] or attr_type == obj_type:
            mod_dic[attr_name] = obj
            setattr(instance, attr_name, obj)
            return
        elif hasattr(attr_type, '__origin__') and attr_type.__origin__ == list and attr_type.__args__[0] == obj_type:
            getattr(instance, attr_name).append(obj)
            return
    

In [ ]:
my_img = Image
add_to_instance(my_img, pix)

In [ ]:
Image.__annotations__

In [ ]:
from src.metagpt.predictors import predictor_tree
from ome_types._autogenerated.ome_2016_06 import Image, Pixels

dependency_tree = predictor_tree.build_tree(Pixels)
state_predicted = dependency_tree.predict_meta(ome_raw)

In [ ]:
from datetime import datetime

datetime.now().isoformat()

In [ ]:




# Example usage:
output_path = 'output.json'
with save_cell_output(output_path):
    # Your code here
    print("This is an example output.qweqweq")
    # Any other code whose output you want to capture


In [ ]:
from src.metagpt.utils import save_and_stream_output
from time import sleep
with save_and_stream_output():
    # Your code here
    print("This is an example output.")
    # Any other code whose output you want to capture
    for i in range(5):
        sleep(1)
        print(f"Line {i + 1}")

In [ ]:
Experimenter.__name__

In [ ]:
from pydantic import BaseModel
from typing import Optional
from ome_types._autogenerated.ome_2016_06 import Image, Pixels, OME, Experimenter

class MaybeExperimenter(BaseModel):
    Experimenter: Optional[Experimenter] = None

In [ ]:
from ome_types._autogenerated.ome_2016_06 import Image, Pixels, OME, Experimenter
with save_and_stream_output():
    exp = Experimenter(id="Exp1")
    model = Experimenter
    object = MaybeExperimenter()
    object.Experimenter = exp
    object = getattr(object, model.__name__)
    print(type(object))

In [ ]:
f = [[]]

for i in f:
    print(i)

In [2]:
from ome_types._autogenerated.ome_2016_06 import Image, Pixels, OME, Experimenter
from src.metagpt.predictors import predictor_tree
from src.metagpt.utils import save_and_stream_output

with save_and_stream_output():
    tree = predictor_tree.build_tree(OME)
    metadata = tree.predict_meta(ome_raw)


Output()

Output()

/home/aaron/Documents/Projects/marvin/src/marvin/utilities/tools.py:176: UserWarning: Unrecognized fields for type <class 'ome_types._autogenerated.ome_2016_06.dataset_ref.DatasetRef'>: {'name', 'description'}
  output = tool.function._python_fn(**arguments)


####################################################################################
Predicting metadata for DatasetRef


Output()

...

####################################################################################
Predicting metadata for Union


Output()

####################################################################################
Predicting metadata for ROI


Output()

####################################################################################
Predicting metadata for OME

Cell output saved to out/jupyter_cell_outputs/cell_output_2024-06-20T17:08:19.084282_.json


In [3]:
metadata

OME(
   plates=[{'id': 'Plate:0'}],
   instruments=[<1 field_type>],
)

In [ ]:
# get the n attributes of an object:
# 
# 
for v in metadata.experimenters[0].__dict__.values():
    if v:
        print(v)

In [ ]:
len([v for v in metadata.experimenters[0].__dict__.values() if v])